In [20]:
import selenium
import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys

In [21]:
driver = webdriver.Chrome(r"D://chromedriver.exe")

In [22]:
driver.maximize_window()

In [23]:
products = ['laptops','mobiles','printers','speakers','home theatres']

In [24]:
# Fetching the website
driver.get("https://www.flipkart.com")

In [25]:
# Closing pop-ups
try:
    driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']").click()
except:
    print("--")

In [26]:
def get_results(driver, prod_name):
    # Entering input item in  search field
    item_inp = driver.find_element_by_xpath("//input[@class='_3704LK']")
    item_inp.send_keys(Keys.CONTROL,"A",Keys.DELETE)
    item_inp.send_keys(prod_name)
    print("Value Entered in Search Box...")
    time.sleep(1)

    # Clicking Search Button
    search_btn =  driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
    search_btn.click()
    print("Search Button Clicked...")
    time.sleep(2)

    # Sorting by popularity
    bttn = driver.find_element_by_xpath("//div[contains(text(),'Popularity')]")
    bttn.click()
    print("Sorting by Popularity...")
    time.sleep(2)

In [27]:
def fetch_url(driver):
#Fetching Product URL for first 100 pages
    url = []

    while len(url)<=100:
        try:
            temp = driver.find_elements_by_xpath('//a[@class="s1Q9rs"]')
            for i in temp:
                url.append(i.get_attribute('href'))
            try:
                next_btn = driver.find_element_by_xpath("//span[contains(text(),'Next')]//parent::a[1]")
                next_btn.click()
                time.sleep(2)
            except NoSuchElementException:
                print("Displayed Results less than three pages")
        except:
            print("Exception")
            break
    print("------------------------------------------URLS FETCHED------------------------------------------")
    print("TOTAL URLS : ", len(url))
    return url

In [28]:
def get_reviews(prod_name, driver, url):
    reviews = pd.DataFrame(columns = ['Product_type', 'Rating', 'Review_title', 'Review_Date', 'Review'])
    count = 0
    for i in url:
        count+=1
        driver.get(i)
        time.sleep(2)

        try:
            all_reviews =  driver.find_element_by_xpath("//div[@class='_3UAT2v _16PBlm']")
            all_reviews.click()
            time.sleep(1)

        except NoSuchElementException as e:
            print("Exception")
            continue
        
        except Exception as e:
            print(e)
            return reviews

        # Expanding reviews
        try:
            read_more =  driver.find_elements_by_xpath("//span[@class='_1BWGvX']")
            for i in read_more:
                i.click()
                time.sleep(0.5)
        except:
            print("No Expand req")
        print('==========================================PRODUCT',count,'==========================================')
        temp_reviews = prod_review_fetch(prod_name, driver)
        if temp_reviews=='Skip':
            continue
        temp_reviews_df = pd.DataFrame(data = temp_reviews, columns = ['Product_type', 'Rating', 'Review_title', 'Review_Date', 'Review'])
        reviews = pd.concat([reviews, temp_reviews_df])
    return reviews

In [29]:
def option_change(driver,j):
    try:
        sel = Select(driver.find_element_by_xpath("//select[@name='sortFilter']"))
    except:
        print(j,"Selection exception")
        return 'break'
    try:
        sel.select_by_value (j)
        time.sleep(2)
    except:
        print(j,"Option exception")
        return 'break'

In [30]:
def prod_review_fetch(prod_name, driver):
    reviews = []
    k=0
    opt = ['NEGATIVE_FIRST', 'MOST_RECENT']
    if option_change(driver,opt[k])=='break':
        return 'Skip'
    while True:
        try:
            review_list = driver.find_elements_by_xpath('//div[@class="_1AtVbE col-12-12"]')
            time.sleep(0.5)
        except NoSuchElementException as e:
            print("Exception No reviews for Product")
            break
        except Exception as e:
            break

        for i in range(3,12):
            path = "//div[@class='_1AtVbE col-12-12']["+str(i)+"]"

            try:
                rating = driver.find_element_by_xpath(path+"//img//parent::div").text
            except Exception as e:
                print('Ratings_Exception')
                print(e)
                continue

            try:
                rev_title = driver.find_element_by_xpath(path+"//p[@class='_2-N8zT']").text
            except Exception as e:
                print('Rev_title_Exception')
                print(e)
                continue

            try:
                revs = driver.find_element_by_xpath(path+"//div[@class='t-ZTKy']").text
            except Exception as e:
                print('Rev_Exception')
                print(e)
                continue

            try:
                rev_date = driver.find_element_by_xpath(path+"//p[@class='_2sc7ZR']").text
            except Exception as e:
                print('Rev_Date_Exception')
                print(e)
                continue

            temp = [prod_name, rating, rev_title, rev_date, revs]
            reviews.append(temp)


        try:
            if k==0 and len(reviews)>=30:
                k=k+1
                if option_change(driver,opt[k])=='break':
                    print("End of Reviews For Product")
                    break
                continue
            elif k==1 and len(reviews)>=45:
                print("End of Reviews For Product")
                break
            next_btn = driver.find_element_by_xpath("//span[contains(text(),'Next')]//parent::a[1]")
            next_btn.click()
            time.sleep(2)
            print("Next Page   : ", opt[k])
        except NoSuchElementException:
            print("End of Reviews For Product")
            break

    return reviews

In [31]:
df = pd.DataFrame(columns = ['Product_type', 'Rating', 'Review_title', 'Review_Date', 'Review'])
df

Empty DataFrame
Columns: [Product_type, Rating, Review_title, Review_Date, Review]
Index: []

In [32]:
for i in products[2:]:
    print('\033[1m','******************************************',i.upper(),'******************************************','\033[0m')
    get_results(driver,i)
    links = fetch_url(driver)
    prod_reviews = get_reviews(i,driver,links)
    df = pd.concat([df, prod_reviews])

 ****************************************** PRINTERS ****************************************** 
Value Entered in Search Box...
Search Button Clicked...
Sorting by Popularity...
------------------------------------------URLS FETCHED------------------------------------------
TOTAL URLS :  120
Exception
Exception
==========================================PRODUCT 3 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 4 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 5 ==========================================
Next Page   :  NEGATIVE_FIRST
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][4]//

Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][11]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

End of Reviews For Product
==========================================PRODUCT 24 ==========================================
Next Page   :  NEGATIVE_FIRST
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][4]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][5]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][6]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

==========================================PRODUCT 39 ==========================================
Next Page   :  NEGATIVE_FIRST
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][8]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][9]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][10]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][11]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

End of Reviews For Product
Exception
==========================================PRODU

Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][7]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][8]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][9]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][10]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE

==========================================PRODUCT 76 ==========================================
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 77 ==========================================
Next Page   :  NEGATIVE_FIRST
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][9]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][10]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][11]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

End of Reviews For Product
==========================================PRODUCT 78 ==========================================
Next 

...

==========================================PRODUCT 6 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 7 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 8 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 9 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 10 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FI

Next Page   :  MOST_RECENT
End of Reviews For Product
==========================================PRODUCT 39 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][11]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Next Page   :  MOST_RECENT
End of Reviews For Product
==========================================PRODUCT 40 ==========================================
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][11]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

End of Reviews For Product
==========================================PRODUCT 41 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
E

Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 62 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 63 ==========================================
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][8]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][9]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][10]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exce

==========================================PRODUCT 82 ==========================================
Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][6]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][7]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][8]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1AtVbE col-12-12'][9]//img//parent::div"}
  (Session info: chrome=91.0.4472.106)

Ratings_Exception
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='_1

Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 104 ==========================================
NEGATIVE_FIRST Selection exception
Exception
==========================================PRODUCT 106 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 107 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
==========================================PRODUCT 108 ==========================================
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
Next Page   :  NEGATIVE_FIRST
End of Reviews For Product
Exception
==========================================PRODUCT 110 ===================================

In [33]:
df

Product_type Rating            Review_title   Review_Date  \
0        printers      1    Utterly Disappointed     Jul, 2017   
1        printers      1  Worst experience ever!     Sep, 2017   
2        printers      1    Utterly Disappointed     Dec, 2017   
3        printers      1     Defective Item Sent     Mar, 2017   
4        printers      1               Very poor     Apr, 2018   
..            ...    ...                     ...           ...   
40  home theatres      5       Worth every penny  2 months ago   
41  home theatres      5      Highly recommended  4 months ago   
42  home theatres      5                  Super!  4 months ago   
43  home theatres      5               Just wow!  4 months ago   
44  home theatres      1        Terrible product  4 months ago   

                                               Review  
0   Excellent and good cartridge.\nWhen I bought 1...  
1   Worst product,, can take only 150 pages but it...  
2   Dont buy this go for tank ink printer, Cheatin...  
3   A defective cartridge was sent. Now I am stuck...  
4   incompatible cartridge error showing for HP 38...  
..                                                ...  
40                                 super good product  
41                                                 Ok  
42                                Very nice nice Bess  
43                                              Super  
44                            Very very worst product  

[10965 rows x 5 columns]

In [34]:
df['Rating'].value_counts()

1    5796
5    2257
4    1488
3     830
2     594
Name: Rating, dtype: int64

In [35]:
df.to_csv('Flipkart_Reviews_P_S_HT.csv', index = False)

In [36]:
driver.close()

In [39]:
df_1 = pd.read_csv("Flipkart_Reviews_L_M.csv")
df_1

Product_type  Rating               Review_title   Review_Date  \
0         laptops       1             Unsatisfactory   20 days ago   
1         laptops       1       Utterly Disappointed  7 months ago   
2         laptops       1  Did not meet expectations  7 months ago   
3         laptops       1          Absolute rubbish!  8 months ago   
4         laptops       1       Utterly Disappointed   1 month ago   
...           ...     ...                        ...           ...   
7886      mobiles       5                  Just wow!    2 days ago   
7887      mobiles       5                    Awesome    2 days ago   
7888      mobiles       5                  Must buy!    2 days ago   
7889      mobiles       5          Terrific purchase    2 days ago   
7890      mobiles       5                  Excellent    2 days ago   

                                                 Review  
0     Battery backup very poor , specified as 6 hour...  
1                         Battery capacity is very low.  
2     after 2 days my windows get crashed and laptop...  
3           Booting speed is slow,but all over good!! 👌  
4     One of the worst laptop I have ever used.it us...  
...                                                 ...  
7886                                               Good  
7887  Awesome, satisfied with mobile. Best mid range...  
7888                                               Good  
7889                                 Awesome product!!!  
7890  Whosoever purchase this please read or see the...  

[7891 rows x 5 columns]

In [38]:
df = pd.concat([df_1, df])
df

Product_type Rating               Review_title   Review_Date  \
0         laptops      1             Unsatisfactory   20 days ago   
1         laptops      1       Utterly Disappointed  7 months ago   
2         laptops      1  Did not meet expectations  7 months ago   
3         laptops      1          Absolute rubbish!  8 months ago   
4         laptops      1       Utterly Disappointed   1 month ago   
..            ...    ...                        ...           ...   
40  home theatres      5          Worth every penny  2 months ago   
41  home theatres      5         Highly recommended  4 months ago   
42  home theatres      5                     Super!  4 months ago   
43  home theatres      5                  Just wow!  4 months ago   
44  home theatres      1           Terrible product  4 months ago   

                                               Review  
0   Battery backup very poor , specified as 6 hour...  
1                       Battery capacity is very low.  
2   after 2 days my windows get crashed and laptop...  
3         Booting speed is slow,but all over good!! 👌  
4   One of the worst laptop I have ever used.it us...  
..                                                ...  
40                                 super good product  
41                                                 Ok  
42                                Very nice nice Bess  
43                                              Super  
44                            Very very worst product  

[18856 rows x 5 columns]

In [40]:
df.to_csv('Flipkart_Reviews.csv', index = False)